In [1]:
# Impordid
import pygame
import random
import sys
import neat

pygame 2.0.1 (SDL 2.0.14, Python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Aluse klass
class Base:
    BASE_X_POS = 0
    BASE_IMG = None

    def __init__(self, img):
        self.BASE_IMG = img

    def draw(self, screen):
        screen.blit(self.BASE_IMG, (self.BASE_X_POS, 700))
        screen.blit(self.BASE_IMG, (self.BASE_X_POS + 575, 700))

    def move(self, screen):
        self.BASE_X_POS -= 1
        self.draw(screen)
        if self.BASE_X_POS <= -575:
            self.BASE_X_POS = 0

In [3]:
# Linnu klass
class Bird:
    GRAVITY = 0.25
    BIRD_MOVEMENT = 0
    BIRD_IMG = None
    BIRD_RECT = None
    JUMP_HEIGHT = 6
    TIMING = 0

    def __init__(self, BIRD_IMG, BIRD_RECT):
        self.BIRD_IMG = BIRD_IMG
        self.BIRD_RECT = BIRD_RECT

    def animation(self):
        if self.TIMING%30 == 0:
            self.BIRD_IMG = pygame.transform.scale2x(pygame.image.load('assets/bird3.png')).convert()
        elif self.TIMING%20 == 0:
            self.BIRD_IMG = pygame.transform.scale2x(pygame.image.load('assets/bird2.png')).convert()
        elif self.TIMING%10 == 0:
            self.BIRD_IMG = pygame.transform.scale2x(pygame.image.load('assets/bird1.png')).convert()
        self.TIMING += 1

    def move(self):
        self.BIRD_MOVEMENT += self.GRAVITY
        self.BIRD_RECT.centery += self.BIRD_MOVEMENT
        self.animation()

    def draw(self, screen):
        screen.blit(self.BIRD_IMG, self.BIRD_RECT)

    def collision(self, pipes):
        if self.BIRD_RECT.top <= -100 or self.BIRD_RECT.bottom >= 900:
            return False
        for pipe in pipes:
            if self.BIRD_RECT.colliderect(pipe.PIPE_TOP) or self.BIRD_RECT.colliderect(pipe.PIPE_BOTTOM):
                return False
        return True

    def jump(self):
        self.BIRD_MOVEMENT = 0
        self.BIRD_MOVEMENT -= self.JUMP_HEIGHT

In [4]:
# Mündi klass
class Coin:
    COIN_IMG = None
    COIN_RECT = None
    is_Gotten = False

    def __init__(self, COIN_IMG, COIN_RECT):
        self.COIN_IMG = COIN_IMG
        self.COIN_RECT = COIN_RECT

    def move(self):
        self.COIN_RECT.centerx -= 3

    def draw(self, screen):
        screen.blit(self.COIN_IMG, self.COIN_RECT)

In [5]:
# Toru klass
class Pipe:
    gap = 175
    PIPE_TOP = None
    PIPE_BOTTOM = None
    passed = False
    next_pipe = False
    height = None

    def __init__(self, img, img_rev):
        self.height = random.randint(300, 625)
        self.IMG = img
        self.IMG_REV = img_rev
        self.PIPE_BOTTOM = self.IMG.get_rect(midtop=(700, self.height))
        self.PIPE_TOP = self.IMG_REV.get_rect(midbottom=(700, self.height - self.gap))

    def move(self):
        self.PIPE_TOP.centerx -= 3
        self.PIPE_BOTTOM.centerx -= 3

    def draw(self, screen):
        screen.blit(self.IMG_REV, self.PIPE_TOP)
        screen.blit(self.IMG, self.PIPE_BOTTOM)

### AI mängib
Praegu AI mängib 50 skoorini, sest selleks ajaks peab olema AI juba päris hea

In [6]:
# ORIGINAL IMAGES FROM https://github.com/samuelcust/flappy-bird-assets
# GAME BASED ON https://www.youtube.com/watch?v=UZg49z76cLw
# AI BASE ON https://www.youtube.com/watch?v=MPFWsRjDmnU&list=PLzMcBGfZo4-lwGZWXz5Qgta_YNX3_vLS2&index=5

WIN_WIDTH = 575
WIN_HEIGHT = 800
high_score = 0

pygame.init()
screen = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
clock = pygame.time.Clock()

# Loeme sisse erinevad pildid
game_font = pygame.font.Font('04B_19.ttf', 40)
BIRD_IMG = pygame.transform.scale2x(pygame.image.load('assets/bird1.png')).convert()
BG_IMG = pygame.transform.scale2x(pygame.image.load('assets/bg.png')).convert()
BASE_IMG = pygame.transform.scale2x(pygame.image.load('assets/base.png')).convert()
PIPE_IMG = pygame.transform.scale2x(pygame.image.load('assets/pipe.png')).convert()
PIPE_IMG_REV = pygame.transform.flip(PIPE_IMG, False, True).convert()
START_GAME_SUFACE = pygame.image.load('assets/message.png').convert_alpha()
STAT_FONT = pygame.font.SysFont("comicsans", 50)
COIN_IMG = pygame.image.load('assets/coin.png')
COIN_IMG = pygame.transform.scale(COIN_IMG, (30,30))


def print_alive(birds):
    # Funktsioon, mis näitab ekraanil, kui palju linde on praegu elus
    alive_surface = STAT_FONT.render(f"Alive: {birds}", True, (255, 255, 255))
    alive_rect = alive_surface.get_rect(center=(70, 20))
    screen.blit(alive_surface, alive_rect)


def score_display(score, high_score):
    # Funktsioon, mis näitab praeguse skoori ja parimat skoori
    score_surface = STAT_FONT.render(f"Score: {score}", True, (255, 255, 255))
    score_rect = score_surface.get_rect(center=(485, 45))
    screen.blit(score_surface, score_rect)

    high_score_surface = STAT_FONT.render(f"High Score: {high_score}", True, (255, 255, 255))
    high_score_rect = high_score_surface.get_rect(center=(440, 20))
    screen.blit(high_score_surface, high_score_rect)


def update_score(pipes, bird):
    # Funktsioon, mis vaatab, kas lin(d/nnud) on läbinud toru esimest korda
    for pipe in pipes:
        if not pipe.passed and pipe.PIPE_BOTTOM.centerx < bird.BIRD_RECT.centerx:
            pipe.passed = True
            return True
    return False


def remove_pipe(pipes):
    # Eemaldame toru, mis on ekraanist välja läinud
    pipes_new = []
    for i in range(len(pipes)):
        if pipes[i].PIPE_BOTTOM.centerx > -50:
            pipes_new.append(pipes[i])
    return pipes_new


def add_pipe(pipes, bird):
    # Lisame toru, kui lind hakkab jõudma toru lähedale
    for pipe in pipes:
        if bird.BIRD_RECT.centerx + 80 > pipe.PIPE_BOTTOM.centerx and not pipe.next_pipe:
            pipe.next_pipe = True
            return True
    return False


def eval(genomes, config):
    # Hindamis funktsioon
    global high_score
    
    COIN_RECT = COIN_IMG.get_rect(center=(-30, 25))

    birds = []
    nets = []
    genes = []
    for g_id, genome in genomes:
        genome.fitness = 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        BIRD_RECT = BIRD_IMG.get_rect(center=(225, 325))
        birds.append(Bird(BIRD_IMG, BIRD_RECT))
        genes.append(genome)

    base = Base(BASE_IMG)
    pipes = [Pipe(PIPE_IMG, PIPE_IMG_REV)]
    score = 0

    game = True
    while game and len(birds) > 0:
        
        ### Muuta see või kustuda ära järgmine if lause, kui tahate, et AI mängiks kauem
        if score > 50:
            break
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        screen.blit(BG_IMG, (0, 0))
        
        coin = Coin(COIN_IMG, COIN_RECT)
        coin.draw(screen)
        coin.move()
        
        # Vaatame, millist torust peame läbima
        pipe_idx = 0
        if len(birds) > 0:
            if len(pipes) > 1 and pipes[0].passed:
                if birds[0].BIRD_RECT.centerx > pipes[0].PIPE_BOTTOM.topright[0] + 30:
                    pipe_idx = 1

        # Läbime kõik linnud ja aktiveerime tanh funktsioone, kui tanh väärtus on suurem kui 0.8 siis hüppame 
        for i, bird in enumerate(birds):
            genes[i].fitness += 0.4
            bird.move()

            # Sisendid
            output = nets[i].activate((bird.BIRD_RECT.y, # linnu y koordinaad
                                       abs(bird.BIRD_RECT.y - pipes[pipe_idx].PIPE_TOP.bottom), # linnu y koordinaad lahutada üleval oleva toru kõige madalam punkt
                                       abs(bird.BIRD_RECT.y - pipes[pipe_idx].PIPE_BOTTOM.top))) # linnud y koordinaad lahutada alumise oleva toru kõige kõrgem punkt

            if output[0] > 0.8:
                bird.jump()

        passed_pipe = False
        for bird in birds:
            # Vaatame, kas linnud on vastu toru või maad vastu läinud
            if not bird.collision(pipes):
                genes[birds.index(bird)].fitness -= 1
                nets.pop(birds.index(bird))
                genes.pop(birds.index(bird))
                birds.pop(birds.index(bird))
            # Vaatame, kas torust oleme läbinud
            if not passed_pipe:
                passed_pipe = update_score(pipes, bird)
            # Vaatame, kas läheneme torule, kui ja, siis lisame toru ja teeme uue mündi
            if add_pipe(pipes, bird):
                if COIN_RECT.centerx < -30:
                    COIN_RECT = COIN_IMG.get_rect(center=(500, random.randint(300, 625)))
                pipes.append(Pipe(PIPE_IMG, PIPE_IMG_REV))
            # Vaatame, kas oleme mündi korjanud
            if bird.BIRD_RECT.colliderect(coin.COIN_RECT):
                if not coin.is_Gotten:
                    score += 2
                    coin.is_Gotten = True
                genes[birds.index(bird)].fitness += 10
                COIN_RECT = COIN_IMG.get_rect(center=(-30, 125))
                
        if len(birds) == 0:
            COIN_RECT = COIN_IMG.get_rect(center=(-30, 125))
        
        # Lisame skoorile juurde, et läbisime toru ja suurendame linnu fitness
        if passed_pipe:
            score += 1
            for g in genes:
                g.fitness += 5

        for bird in birds:
            bird.draw(screen)

        for pipe in pipes:
            pipe.move()
            pipe.draw(screen)

        print_alive(len(birds))
        score_display(score, high_score)

        pipes = remove_pipe(pipes)

        base.move(screen)

        pygame.display.update()
        clock.tick(100)
    if score > high_score:
        high_score = score
    print(f"High score {high_score}")


def run(config_file):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet,
                                neat.DefaultStagnation, config_file)
    pop = neat.Population(config)
    pop.add_reporter(neat.StdOutReporter(True))
    pop.add_reporter(neat.StatisticsReporter())
    winner = pop.run(eval, 50)


if __name__ == '__main__':
    run('neat-config.txt')


 ****** Running generation 0 ****** 

High score 22
Population's average fitness: 108.44000 stdev: 239.83213
Best fitness: 1111.80000 - size: (1, 3) - species 1 - id 12
Average adjusted fitness: 0.076
Mean genetic distance 1.173, standard deviation 0.326
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    20   1111.8    0.076     0
Total extinctions: 0
Generation time: 26.126 sec

 ****** Running generation 1 ****** 

High score 22
Population's average fitness: 38.98000 stdev: 13.88264
Best fitness: 62.60000 - size: (1, 3) - species 1 - id 12
Average adjusted fitness: 0.351
Mean genetic distance 1.114, standard deviation 0.439
Population of 20 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    20     62.6    0.351     1
Total extinctions: 0
Generation time: 1.645 sec (13.885 average)

 ****** Running generation 2 ****** 

High scor

### Saab ise mängida

In [ ]:
# ORIGINAL IMAGES FROM https://github.com/samuelcust/flappy-bird-assets
# GAME BASED ON https://www.youtube.com/watch?v=UZg49z76cLw
# AI BASE ON https://www.youtube.com/watch?v=MPFWsRjDmnU&list=PLzMcBGfZo4-lwGZWXz5Qgta_YNX3_vLS2&index=5

WIN_WIDTH = 575
WIN_HEIGHT = 800
GAME_ACTIVE = False
score = 0
high_score = 0


def score_display(game_over):
    # Funktsioon, mis näitab praeguse skoori ja parimat skoori
    if not game_over:
        score_surface = game_font.render(str(int(score)), True, (255, 255, 255))
        score_rect = score_surface.get_rect(center=(288, 100))
        screen.blit(score_surface, score_rect)

    high_score_surface = high_score_font.render('High score: ' + str(high_score), True, (255, 255, 255))
    high_score_rect = high_score_surface.get_rect(center=(288, 60))
    screen.blit(high_score_surface, high_score_rect)


def update_score():
    # Funktsioon, mis vaatab, kas lin(d/nnud) on läbinud toru esimest korda
    for pipe in pipes:
        if not pipe.passed and pipe.PIPE_BOTTOM.centerx < bird.BIRD_RECT.centerx:
            pipe.passed = True
            return True
    return False


def remove_pipe():
    # Eemaldame toru, mis on ekraanist välja läinud
    pipes_new = []
    for i in range(len(pipes)):
        if pipes[i].PIPE_BOTTOM.centerx > -50:
            pipes_new.append(pipes[i])
    return pipes_new


def add_pipe():
    # Lisame toru, kui lind hakkab jõudma toru lähedale
    for pipe in pipes:
        if bird.BIRD_RECT.centerx + 80 > pipe.PIPE_BOTTOM.centerx and not pipe.next_pipe:
            pipe.next_pipe = True
            return True
    return False


if __name__ == '__main__':
    pygame.init()
    screen = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    clock = pygame.time.Clock()

    game_font = pygame.font.Font('04B_19.ttf', 40)
    high_score_font = pygame.font.Font('04B_19.TTF', 18)

    BIRD_IMG = pygame.transform.scale2x(pygame.image.load('assets/bird1.png')).convert()
    BG_IMG = pygame.transform.scale2x(pygame.image.load('assets/bg.png')).convert()
    BASE_IMG = pygame.transform.scale2x(pygame.image.load('assets/base.png')).convert()
    BIRD_RECT = BIRD_IMG.get_rect(center=(225, 325))
    PIPE_IMG = pygame.transform.scale2x(pygame.image.load('assets/pipe.png')).convert()
    PIPE_IMG_REV = pygame.transform.flip(PIPE_IMG, False, True).convert()
    START_GAME_SUFACE = pygame.image.load('assets/message.png').convert_alpha()
    START_GAME_REC = START_GAME_SUFACE.get_rect(center=(288, 400))
    COIN_IMG = pygame.image.load('assets/coin.png')
    COIN_IMG = pygame.transform.scale(COIN_IMG, (30,30))
    COIN_RECT = COIN_IMG.get_rect(center=(-30, 25))

    pipes = [Pipe(PIPE_IMG, PIPE_IMG_REV)]

    bird = Bird(BIRD_IMG, BIRD_RECT)

    base = Base(BASE_IMG)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and GAME_ACTIVE:
                    bird.jump()
                if event.key == pygame.K_SPACE and GAME_ACTIVE is False:
                    GAME_ACTIVE = True
                    COIN_RECT = COIN_IMG.get_rect(center=(-30, 125))
                    BIRD_RECT = BIRD_IMG.get_rect(center=(225, 325))
                    bird = Bird(BIRD_IMG, BIRD_RECT)
                    pipes = [Pipe(PIPE_IMG, PIPE_IMG_REV)]
                    score = 0

        screen.blit(BG_IMG, (0, 0))

        if GAME_ACTIVE:
            bird.move()
            GAME_ACTIVE = bird.collision(pipes)

            coin = Coin(COIN_IMG, COIN_RECT)
            coin.draw(screen)
            coin.move()

            for pipe in pipes:
                pipe.move()

            score_display(False)
            passed = update_score()
            if passed:
                score += 1
            if add_pipe():
                if COIN_RECT.centerx < -30:
                    COIN_RECT = COIN_IMG.get_rect(center=(500, random.randint(300, 625)))
                pipes.append(Pipe(PIPE_IMG, PIPE_IMG_REV))
            if bird.BIRD_RECT.colliderect(coin.COIN_RECT):
                score += 2
                COIN_RECT = COIN_IMG.get_rect(center=(-30, 125))
            
            pipes = remove_pipe()
            
            for pipe in pipes:
                pipe.draw(screen)
            
            bird.draw(screen)

        else:
            screen.blit(START_GAME_SUFACE, START_GAME_REC)
            if score > high_score:
                high_score = score
            score_display(True)


        base.move(screen)

        pygame.display.update()
        clock.tick(100)